# 2.1.1 Preconditioners in NGSolve

Preconditioners are approximate inverses which are used within iterative methods to solve linear or non-linear equations.

Here are some built-in preconditioners in NGSolve:

* Jacobi (`local`) and block Jacobi 
* Direct solvers using sparse factorization (`direct`)
* Geometric multigrid with different block-smoothers (`multigrid`)
* Algebraic multigrid preconditioner (`h1amg`)
* p-version element-level BDDC (`bddc`)

This tutorial quickly shows how to use some of these within a solver.

In [ ]:
from ngsolve import *
import matplotlib.pyplot as plt

### A simple test problem 

In order to experiment with various preconditioners, 
let us define a simple Poisson solver with the name
of a preconditioner as the keyword argument `precond`.

In [ ]:
def SolveProblem(h=0.5, p=1, levels=1, 
                 condense=False,
                 precond="local"):
    """
    Solve Poisson problem on l refinement levels.
    PARAMETERS:
        h: coarse mesh size
        p: polynomial degree 
        l: number of refinement levels
        condense: if true, perform static condensations
        precond: name of a built-in preconditioner
    Returns: (ndof, niterations)
        List of tuples of number of degrees of freedom and iterations
    """
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))
    # mesh = Mesh(unit_cube.GenerateMesh(maxh=h))
    fes = H1(mesh, order=p, dirichlet="bottom|left")
    
    u, v = fes.TnT() 
    a = BilinearForm(grad(u)*grad(v)*dx, condense=condense)
    f = LinearForm(v*dx)
    gfu = GridFunction(fes)
    Draw(gfu)
    c = Preconditioner(a, precond) # Register c to a BEFORE assembly

    steps = []
    
    for l in range(levels):
        if l > 0: 
            mesh.Refine()
        fes.Update()
        gfu.Update()

        with TaskManager():
            a.Assemble()
            f.Assemble()

            # Conjugate gradient solver
            inv = CGSolver(a.mat, c.mat, maxsteps=1000)

            # Solve steps depend on condense 
            if condense:
                f.vec.data += a.harmonic_extension_trans * f.vec
            
            gfu.vec.data = inv * f.vec
            
            if condense:
                gfu.vec.data += a.harmonic_extension * gfu.vec
                gfu.vec.data += a.inner_solve * f.vec
        steps.append ( (fes.ndof, inv.GetSteps()) )
        if fes.ndof < 15000:
            Redraw()
    return steps

The `Preconditioner` registers itself to the `BilinearForm`. Whenever the `BilinearForm` is re-assembled, the `Preconditioner` is updated as well.

### The `local` preconditioner 

The `local` preconditioner is a simple Jacobi preconditioner. 
The number of CG-iterations with the local preconditioner is proportional to $h^{-1} \sim 2^l$:


In [ ]:
SolveProblem(precond="local")

In [ ]:
res_local = SolveProblem(levels=9, precond="local")
res_local

### Multigrid preconditioner 

A geometric multigrid `Preconditioner` uses the sequence of refined meshes to define a preconditioner yielding optimal iteration numbers (and complexity). It uses a direct solve on the coarsest level, and block Gauss-Seidel smoothers on the refined levels.

In [ ]:
res_mg = SolveProblem(levels=9, precond="multigrid")
res_mg

In [ ]:
plt.xscale("log")
plt.yscale("log")
plt.plot(*zip(*res_local), "-*")
plt.plot(*zip(*res_mg), "-+")
plt.legend(['local', 'mg'])
plt.show()

###  Multigrid implementation for higher order spaces 

For high order finite element spaces NGSolve uses hierarchical bases.
Let the (small) sub-spaces $V_E$, $V_F$, and $V_C$ be those generated by basis functions associated with an edge $E$, a face $F$, and a cell $C$, respectively. Then the full
space admits the decomposition

$$
V_{hp} = V_{p=1} + \sum_{\text{edges }E} V_E + \sum_{\text{faces }F} V_F + \sum_{\text{cells }C} V_C
$$

where $V_{p=1}$ refers to the lowest order finite element subspace.
The system matrix then takes the block form

$$
A = \left( \begin{array}{cccc}
A_{VV} & A_{VE} & A_{VF} & A_{VC} \\
A_{EV} & A_{EE} & A_{EF} & A_{EC} \\
A_{FV} & A_{FE} & A_{FF} & A_{FC} \\
A_{CV} & A_{CE} & A_{CF} & A_{CC} \\
\end{array} \right)
$$

where the $A_{VV}$-block is exactly the system matrix of a lowest order method, representing the part of operator acting on the $V_{p=1}$ subspace.

NGSolve's *multigrid implementation for a high order method uses h-version multigrid for the lowest order block,* and  local block-smoothing for the high order bubble basis functions.

In [ ]:
for p in range(1,10):
    r = SolveProblem(h=0.5, p=p, levels=4, condense=False, 
                     precond="multigrid")
    print ("p=", p, ": ndof,nsteps=", r)          

We observe that the number of iterations grows mildly with the degree $p$ while remaining  bounded with mesh refinement.

Performing static condensation improves the situation:

In [ ]:
for p in range(1,10):
    r = SolveProblem(h=0.5, p=p, levels=4, condense=True, 
                     precond="multigrid")
    print ("p=", p, ": ndof,nsteps=", r)       

### Element-wise BDDC preconditioner

A built-in element-wise BDDC (Balancing Domain Decomposition preconditioner with Constraints) preconditioner is also available. In contrast to local or multigrid preconditioners, the BDDC preconditioner needs access to the element matrices. This is exactly why we need to register the preconditioner with the bilinear form `bfa` before calling `bfa.Assemble()`.

In [ ]:
for p in range(1,10):
    r = SolveProblem(h=0.5, p=p, levels=4, condense=True, 
                     precond="bddc")
    print ("p=", p, ": ndof,nsteps=", r)  

The BDDC preconditioner needs more iterations, but the work per iteration is less, so performance is similar to multigrid. **This element-wise BDDC preconditioner works well in shared memory parallel as well as in distributed memory mode.** See $\S$[2.1.4](../unit-2.1.4-bddc/bddc.ipynb) for more about BDDC preconditioner and how to combine it with an algebraic multigrid coarse solver. 